In [ ]:
#NO_WINDOWS_SUPPORT : https://stackoverflow.com/questions/52932406/is-the-class-generator-inheriting-sequence-thread-safe-in-keras-tensorflow

In [1]:
#https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly#data-generator
#https://keras.io/utils/

# Expected usage:
# train_gen=DataGenerator(X_train,Y_train,**params)
# val_gen=DataGenerator(X_val,Y_val,**params)
# model.fit_generator(generator=train_gen, 
#                     validation_data=val_gen ,
#                     use_multiprocessing=True, 
#                     workers=2)

In [2]:
import numpy as np
import tensorflow as tf
#import keras
from unet import *
from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adam

C:\Anaconda\envs\py36gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
#Every Sequence must implement the __getitem__ and the __len__ methods. 
#If you want to modify your dataset between epochs you may implement on_epoch_end. 




class DataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self,X,Y,batch_size=32,shuffle=False):
        self.X=X
        self.Y=Y
        self.batch_size=batch_size
        self.shuffle=shuffle
        self.on_epoch_end()
        
        
    def __len__(self):
        return int(np.floor(len(self.X)/self.batch_size))
        

    def __getitem__(self,index):
        #The method __getitem__ should return a complete batch.
        
        batch_x=self.X[index*self.batch_size:(index+1)*self.batch_size]
        batch_y=self.Y[index*self.batch_size:(index+1)*self.batch_size]
        
        return np.array(batch_x),np.array(batch_y)
        
    

In [4]:
def create_data(no_samples=100,dx=256,dy=256,dz=1):    
    #create training data.
    X=np.random.randn(no_samples,dx,dy,dz)
    #generate reference data (label/true data)
    Y=0.8*X+2
    print(X.shape,Y.shape)
    #generate some validation data
    X_val=np.random.randn(no_samples,dx,dy,dz)
    Y_val=0.8*X_val+2
    print(X_val.shape,Y_val.shape)
    return (X,Y,X_val,Y_val)

In [5]:
X,Y,X_val,Y_val=create_data(no_samples=1000)

(1000, 256, 256, 1) (1000, 256, 256, 1)
(1000, 256, 256, 1) (1000, 256, 256, 1)


In [6]:
train_gen=DataGenerator(X=X,Y=Y,batch_size=32,shuffle=False)

In [7]:
val_gen=DataGenerator(X=X_val,Y=Y_val,batch_size=32,shuffle=False)

In [8]:

def build_model():
    input_tensor=Input((256,256,1))
    model=unet(input_tensor,maxpool=False)
    return model

In [9]:
model=build_model()
model.compile(optimizer=Adam(),loss='mse',metrics=['accuracy'])

(None, 128, 128, 16) (None, 128, 128, 16)
(None, 64, 64, 32) (None, 64, 64, 32)
(None, 32, 32, 64) (None, 32, 32, 64)
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 8, 8, 256) (None, 8, 8, 256)
(None, 4, 4, 512) (None, 4, 4, 512)
ENCDEC
before concat: (deconv.shape,to_conc.shape) :  (None, 8, 8, 256) (None, 8, 8, 256)
after concat: deconv.shape   (None, 8, 8, 512)
(None, 8, 8, 512)
before concat: (deconv.shape,to_conc.shape) :  (None, 16, 16, 128) (None, 16, 16, 128)
after concat: deconv.shape   (None, 16, 16, 256)
(None, 16, 16, 256)
before concat: (deconv.shape,to_conc.shape) :  (None, 32, 32, 64) (None, 32, 32, 64)
after concat: deconv.shape   (None, 32, 32, 128)
(None, 32, 32, 128)
before concat: (deconv.shape,to_conc.shape) :  (None, 64, 64, 32) (None, 64, 64, 32)
after concat: deconv.shape   (None, 64, 64, 64)
(None, 64, 64, 64)
before concat: (deconv.shape,to_conc.shape) :  (None, 128, 128, 16) (None, 128, 128, 16)
after concat: deconv.shape   (None, 128, 128, 32)
(None, 128, 12

In [15]:
print(tf.keras.__version__,tf.__version__)

2.2.4-tf 2.1.0


In [17]:
import math
model.fit_generator(train_gen,
                    steps_per_epoch=len(X)/32
                   #validation_data=val_gen
                   )

  ...
    to  
  ['...']
Train for 31.25 steps
 1/31 [..............................] - ETA: 5:23

ResourceExhaustedError:  OOM when allocating tensor with shape[16] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/batch_normalization/FusedBatchNormV3 (defined at <ipython-input-10-7df35dcfead7>:2) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_distributed_function_4227]

Function call stack:
distributed_function
